In [1]:
import sqlite3, os, sys
import pandas as pd
import tensorflow as tf
import numpy as np
from tqdm.notebook import tqdm

In [2]:
db_path = "db_files/rasmus_classification_muon_3neutrino_3mio.db"

In [3]:
conn = sqlite3.connect(db_path)

In [4]:
cursor = conn.cursor()

In [5]:
cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table';")

In [6]:
print(cursor.fetchall())

[('features',), ('truth',)]


In [7]:
pd.read_sql("select * from features where SRTInIcePulses = 1 limit 100", conn)

,row,event_no,string,dom,pmt,dom_x,dom_y,dom_z,pmt_x,pmt_y,pmt_z,pmt_area,pmt_type,time,charge_log10,lc,pulse_width,SplitInIcePulses,SRTInIcePulses
0,3,120102888,84,51,0,0.063224,-0.295912,-0.468294,0.0,0.0,-1.0,0.0444,20,-0.053353,-0.385923,0,8,1,1
1,6,120102888,84,24,0,0.063224,-0.295912,-0.087726,0.0,0.0,-1.0,0.0444,20,-0.020748,-1.656923,0,8,1,1
2,7,120102888,84,24,0,0.063224,-0.295912,-0.087726,0.0,0.0,-1.0,0.0444,20,-0.032234,0.444979,0,8,1,1
3,9,120102888,79,36,0,-0.041947,-0.159392,-0.252298,0.0,0.0,-1.0,0.0444,20,-0.300482,0.000000,1,1,1,1
4,10,120102888,79,35,0,-0.041947,-0.159392,-0.238220,0.0,0.0,-1.0,0.0444,20,-0.300852,0.892594,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,419,120102901,82,19,0,0.264813,0.259593,-0.015968,0.0,0.0,-1.0,0.0444,20,-0.300482,0.940601,1,1,1,1
96,420,120102901,82,19,0,0.264813,0.259593,-0.015968,0.0,0.0,-1.0,0.0444,20,-0.298629,3.382514,1,1,1,1
97,421,120102901,82,19,0,0.264813,0.259593,-0.015968,0.0,0.0,-1.0,0.0444,20,-0.295295,-0.280277,1,1,1,1
98,422,120102901,82,19,0,0.264813,0.259593,-0.015968,0.0,0.0,-1.0,0.0444,20,-0.291960,0.378965,1,1,1,1


In [8]:
pd.read_sql("select * from truth limit 100", conn)

,event_no,energy_log10,time,position_x,position_y,position_z,direction_x,direction_y,direction_z,azimuth,zenith,pid,interaction_type,muon_track_length,stopped_muon
0,120102888,-1.715544,0.382094,-0.015618,-0.226383,-0.122905,-0.411492,0.898307,-0.036521,0.619221,-0.037939,-12,1,None,None
1,120102889,-1.774794,0.356243,-0.014141,-0.029667,-0.158623,0.048196,-0.056711,1.604578,-0.297473,1.872912,12,1,None,None
2,120102890,-2.208928,0.575249,0.312203,0.264803,-0.211447,-0.775490,0.084519,1.364123,0.938828,1.295564,12,1,None,None
3,120102891,-1.937168,0.373180,0.112214,-0.388287,-0.189285,-0.774064,0.736371,0.070639,0.736760,0.070608,12,1,None,None
4,120102892,-1.745595,0.378069,0.327033,-0.074377,-0.084979,-0.979775,-0.219489,-0.028706,-0.928818,-0.029727,12,1,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,120102996,-1.854441,0.400850,-0.239345,0.215069,-0.132002,1.068348,-0.318197,1.036546,-0.099472,0.933506,12,1,None,None
96,120102997,-1.673846,0.301952,-0.151598,0.031544,-0.255451,0.062935,0.829049,1.324317,0.464113,1.243985,12,1,None,None
97,120102998,-1.765481,0.377518,-0.001261,-0.220404,-0.174025,-0.722768,0.301400,1.360284,0.849843,1.290437,12,1,None,None
98,120102999,-1.664930,0.382847,-0.271504,0.235681,-0.188133,1.227452,-0.223173,0.598145,-0.064644,0.541734,-12,1,None,None


In [10]:
n_data = 100
start_id = conn.execute(f"select distinct event_no from truth where pid = 13")
stop_id  = conn.execute(f"select distinct event_no from truth where pid = 13 limit 1 offset {n_data}")

In [11]:
start_id.fetchall()

[(138674340,),
 (138674341,),
 (138674342,),
 (138674343,),
 (138674344,),
 (138674345,),
 (138674346,),
 (138674347,),
 (138674348,),
 (138674351,),
 (138674352,),
 (138674353,),
 (138674354,),
 (138674355,),
 (138674358,),
 (138674359,),
 (138674360,),
 (138674361,),
 (138674362,),
 (138674363,),
 (138674364,),
 (138674366,),
 (138674367,),
 (138674368,),
 (138674369,),
 (138674370,),
 (138674372,),
 (138674374,),
 (138674375,),
 (138674376,),
 (138674377,),
 (138674378,),
 (138674382,),
 (138674383,),
 (138674384,),
 (138674385,),
 (138674386,),
 (138674388,),
 (138674389,),
 (138674390,),
 (138674391,),
 (138674392,),
 (138674393,),
 (138674394,),
 (138674395,),
 (138674396,),
 (138674398,),
 (138674399,),
 (138674400,),
 (138674401,),
 (138674403,),
 (138674404,),
 (138674405,),
 (138674406,),
 (138674409,),
 (138674410,),
 (138674412,),
 (138674413,),
 (138674414,),
 (138674415,),
 (138674417,),
 (138674418,),
 (138674420,),
 (138674421,),
 (138674422,),
 (138674424,),
 (13867442

In [12]:
stop_id.fetchall()

[(138674472,)]

In [16]:
start, stop = 0, 100000
# start_id  = conn.execute(f"select distinct event_no from features where pid = 13 limit ")
event_nos = pd.read_sql(f"select distinct event_no from features limit {stop} offset {start}", conn)

In [17]:
event_nos

,event_no
0,120102888
1,120102889
2,120102890
3,120102891
4,120102892
...,...
99995,120228027
99996,120228028
99997,120228029
99998,120228030


In [23]:
start, stop = 0, 100000
select_rows = ["dom_x", "dom_y", "dom_z", "time", "charge_log10"]

start_id = conn.execute(f"select distinct event_no from features limit 1 offset {start}").fetchall()[0][0]
stop_id  = conn.execute(f"select distinct event_no from features limit 1 offset {stop}").fetchall()[0][0]

In [24]:
features = ["dom_x", "dom_y", "dom_z", "time", "charge_log10"]
targets  = ["energy_log10", "position_x", "position_y", "position_z", "direction_x", "direction_y", "direction_z"]
feature_call = ", ".join(features)
target_call  = ", ".join(targets)
df_event = pd.read_sql(f"select event_no       from features where event_no >= {start_id} and event_no < {stop_id}", conn)
df_feat  = pd.read_sql(f"select {feature_call} from features where event_no >= {start_id} and event_no < {stop_id}", conn)
df_targ  = pd.read_sql(f"select {target_call } from truth    where event_no >= {start_id} and event_no < {stop_id}", conn)

In [25]:
idx_list    = np.array(df_event)
x_not_split = np.array(df_feat)

In [30]:
np.shape(idx_list)

(5965924, 1)

In [31]:
ids, idx, counts = np.unique(idx_list.flatten(), return_index = True, return_counts = True) 

In [29]:
np.sum(counts)

5965924

In [34]:
x_not_split.shape

(5965924, 5)

In [35]:
xs = np.split(x_not_split, np.cumsum(counts))

In [61]:
ys = np.array(df_targ)

In [43]:
np.shape(xs)

C:\Users\chris\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


(100001,)

In [24]:
for x, y in zip(xs, ys):
    print(x, y)
    break

[[-1.84428951 -1.22482404  0.39792449 -1.56428307  0.16261479]
 [-0.2078301  -0.18691354 -0.44418075 -1.68247499 -1.92155384]
 [ 0.06322446 -0.29591152 -0.46829432 -0.05335309 -0.38592265]
 [ 0.06322446 -0.29591152 -0.14411839  0.90848462  0.23806267]
 [ 0.06322446 -0.29591152 -0.14411839 -1.41830308  1.49374495]
 [ 0.06322446 -0.29591152 -0.08772599 -0.02074843 -1.65692291]
 [ 0.06322446 -0.29591152 -0.08772599 -0.03223416  0.44497857]
 [ 0.          0.29478052 -0.12298133  0.44979622 -0.38592265]
 [-0.04194698 -0.15939176 -0.25229772 -0.30048166  0.        ]
 [-0.04194698 -0.15939176 -0.23821973 -0.30085217  0.89259397]
 [-0.04194698 -0.15939176 -0.2241216  -0.29529455 -1.0557719 ]
 [-0.04194698 -0.15939176 -0.1818273  -0.26083735 -0.62103541]
 [-0.07899002  2.28041226  1.09275381  1.07373101  0.50835111]
 [ 0.74159036  1.95723031  0.97476017  1.00629863  0.68454405]
 [-0.25763961  1.79339814  0.32990768  0.42645424  0.31007972]
 [-0.25763961  1.79339814  0.32990768 -1.44942571  0.08

In [63]:
from sklearn.neighbors import kneighbors_graph as knn
from spektral.data import Graph
neighbors=6

In [65]:
print("Generating adjacency matrices")
graph_list = []
for x, y in tqdm(zip(xs, ys), total = len(xs)):
    a = knn(x[:, :3], neighbors)



    graph_list.append(Graph(x = x, a = a, y = y))

graph_list = np.array(graph_list, dtype = object)

Generating adjacency matrices


In [51]:
s = [x.shape[0] for x in xs]

In [ ]:
xs = np.array(xs, dtype = object)[np.argsort(s)]

In [57]:
size, idx = np.unique(np.sort(s), return_index = True)

In [ ]:
preprocess(list(xs[4: 9]))

In [53]:
np.sort(s)

array([ 14,  19,  19, ..., 822, 876, 913])

In [46]:
idx

array([    0,     1,     4,     9,    21,    45,    85,   143,   244,
         401,   652,  1016,  1483,  2097,  2924,  4001,  5282,  6731,
        8466, 10599, 13093, 15805, 18756, 21801, 24937, 28170, 31567,
       34857, 38137, 41355, 44535, 47573, 50422, 53228, 55770, 58086,
       60383, 62484, 64403, 66274, 67917, 69509, 70992, 72361, 73664,
       74887, 75978, 76999, 77932, 78771, 79617, 80384, 81125, 81818,
       82484, 83089, 83658, 84237, 84756, 85227, 85712, 86149, 86570,
       86957, 87318, 87719, 88092, 88425, 88746, 89059, 89334, 89610,
       89868, 90105, 90354, 90606, 90834, 91059, 91268, 91484, 91711,
       91916, 92108, 92283, 92486, 92656, 92805, 92975, 93132, 93295,
       93449, 93584, 93732, 93878, 94005, 94124, 94236, 94346, 94449,
       94538, 94645, 94733, 94844, 94928, 95026, 95123, 95213, 95306,
       95396, 95483, 95584, 95668, 95763, 95835, 95906, 95966, 96052,
       96127, 96195, 96259, 96320, 96387, 96454, 96511, 96563, 96613,
       96675, 96739,